In [5]:
import requests
import os
import json
import pandas as pd
import numpy as np
import pickle as pkl
import TwitterUtils as TU
import seaborn as sns
import re

# import spacy
# import spacy_langdetect as sld

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")

def generate(instruction, knowledge, dialog):
    if knowledge != '':
        knowledge = '[KNOWLEDGE] ' + knowledge
    dialog = ' EOS '.join(dialog)
    query = f"{instruction} [CONTEXT] {dialog} {knowledge}"
    input_ids = tokenizer(f"{query}", return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=128, min_length=8, top_p=0.9, do_sample=True)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output

# Instruction for a chitchat task
instruction = f'Instruction: given a dialog context, you need to response empathically.'
# Leave the knowldge empty
knowledge = ''
dialog = [
    'Does money buy happiness?',
    'It is a question. Money buys you a lot of things, but not enough to buy happiness.',
    'What is the best way to buy happiness ?'
]
response = generate(instruction, knowledge, dialog)
print(response)

Money buys you good things.


In [9]:
convo_df = pd.read_csv("convo_data.csv")

In [10]:
convo_df.iloc[42].clean_convo

'[3]: https://t.co/xRm5O7VaCt || [0]: Respect my goat \U0001fae1 https://t.co/qtWbliK8Nt || [1]: Tatum sucks || [2]: Respect yourself || [3]: https://t.co/cr47l4df9P || [0]: 😭 ||'

In [97]:
dialog = convo_df.iloc[0].clean_convo.split('||')

In [98]:
dialog

["[6]: First of all, believing is not a scientific mindset. Don't believe even science. Just try to know, whether it is science or astrology. ",
 ' [2]: Also, the person you admire in investing field believes in Vedic astrology. He doesn’t just believe in it, he has actually studied it. I would admire your skepticism if you call it out after studying it or else your opinion will be a ill informed one. ',
 " [7]: No, it didn't impressed me if that was the motive.\nHope U don't carry this at Ur work place. I personally avoid doing biz with clients having such mindset. ",
 " [0]: Namaste sir.  It is OK if you aren't impressed.  Have a good day ahead. ",
 ' [1]: If someone asks whether stock market is gambling or not can you answer him without explaining the context. Similarly you need to understand astrology to get your answers. \n\nI feel like you started getting training from Raga ',
 ' [3]: Pranaam Sir🙏\nWhat is your take on timed muhurat cesarean. Should one go for it? ',
 ' [5]: sir,

In [112]:
# Instruction for a chitchat task
instruction = f'Instruction: given a dialog context, respond in the style of the dialog:'
# Leave the knowldge empty
knowledge = convo_df.iloc[0].country
print(knowledge)
response = generate(instruction, knowledge, dialog)
print(response)

India
BJP is leader of the BJP and BJP is the only candidate.
